In [28]:
import pandas as pd
import math
from scipy import stats
import time
import numpy as np
import os
import plotly.express as px
import matplotlib.pyplot as plt
import datetime

In [11]:
def clean(df):
    """Filter Data 
    If bid or ask = 0 , drop the row
    Else if open insterest = 0, drop the row
    Else if volume = 0, drop the row
    Then keep rest

    Args:
        df (dataframe): option dataframe

    Returns:
        DataFrame : filtered dataframe
    """
    # Drop rows with 0 bid or ask
    df.dropna(inplace=True);
    org = df.shape[0]
    df = df[df.settlement != 0]
    #df = df[df.sett != 0]
    #df = df[df.ask != 0]
    new = df.shape[0]
    print(f"{'=' * 10} Cleaning Data 1 {'=' * 10}")
    print(f"{org - new} rows are dropped")
    # Drop open insterest = 0
    org = df.shape[0]
    df = df[df['open_interest'] != 0]
    new = df.shape[0]
    print(f"{'=' * 10} Cleaning Data 2 {'=' * 10}")
    print(f"{org - new} rows are dropped")
    # Drop volume = 0
    org = df.shape[0]
    df = df[df.volume != 0]
    new = df.shape[0]
    print(f"{'=' * 10} Cleaning Data 3 {'=' * 10}")
    print(f"{org - new} rows are dropped")
    print(f"{df.shape[0]} rows remain")
    
    # trans 
    df.loc[:, 'call_put'] = df.call_put.str.lower()
    df['Date'] = pd.to_datetime(df.Date)
    df['opt_expiration_date'] = pd.to_datetime(df.opt_expiration_date)
    
    df.reset_index(drop=True, inplace=True)
    return df

def get_specific_data(df, is_call, is_Amercian):
    """return specific dataframe according to is_call and is_Amercian

    Args:
        df (dataframe): option data
        is_call (bool): True if call, False if put
        is_Amercian (bool): True if Amercian, False if European
    
    Returns:
        dataframe: specific dataframe
    """
    
    if is_Amercian:
        if is_call:
            return df.loc[(df.call_put == 'C') & (df['style'] == 'A')]
        elif is_call:
            return df.loc[(df.call_put == 'P') & (df['style'] == 'A')]
    else:
        if is_call:
            return df.loc[(df.call_put == 'C') & (df['style'] == 'E')]
        else:
            return df.loc[(df.call_put == 'P') & (df['style'] == 'E')]
        
"""
def get_info(df):
    return info of dataframe
    C_A_num = get_specific_data(df, True, True)
    P_A_num = get_specific_data(df, False, True)
    C_E_num = get_specific_data(df, True, False)
    P_E_num = get_specific_data(df, False, False)
    print(f"{'=' * 10} Info {'=' * 10}")
    print(f"American Call: {C_A_num.}")
    print(f"American Put: {P_A_num}")
    print(f"European Call: {C_E_num}")
    print(f"European Put: {P_E_num}")
    print(f"American: {C_A_num + P_A_num}")
    print(f"European: {C_E_num + P_E_num}")
    """

'\ndef get_info(df):\n    return info of dataframe\n    C_A_num = get_specific_data(df, True, True)\n    P_A_num = get_specific_data(df, False, True)\n    C_E_num = get_specific_data(df, True, False)\n    P_E_num = get_specific_data(df, False, False)\n    print(f"{\'=\' * 10} Info {\'=\' * 10}")\n    print(f"American Call: {C_A_num.}")\n    print(f"American Put: {P_A_num}")\n    print(f"European Call: {C_E_num}")\n    print(f"European Put: {P_E_num}")\n    print(f"American: {C_A_num + P_A_num}")\n    print(f"European: {C_E_num + P_E_num}")\n    '

In [12]:
org = '/Users/dylan/iCollections/桌面文件/Repo/Option_Pricing/Data/option_data/usa_fut_nbbo_2020/'
_ = []
for file_name in os.listdir(org):
    if 'DS' in file_name:
        continue
    if os.path.exists('/Users/dylan/iCollections/桌面文件/Repo/Option_Pricing/Data/Output_data/' + '2020_whole.csv'):
        print('file exists')
        df = pd.read_csv('/Users/dylan/iCollections/桌面文件/Repo/Option_Pricing/Data/Output_data/' + '2020_whole.csv')
        df =clean(df)
        df.drop(df.columns[0], axis=1, inplace=True)
    else:
        print('file not exists')
        output_dir = '/Users/dylan/iCollections/桌面文件/Repo/Option_Pricing/Data/Output_data/'
        input_dir = os.path.join(org, file_name)
        filenames = os.listdir(input_dir)
        index = 0
        dfs = []
        for name in filenames:
            if 'USA' in name:
                print(f"已合并表{index}")
                df_ = pd.read_csv(os.path.join(input_dir, name))
                df_.columns = ['Date', 'underlying', 'exchange', 'root_symbol', 'futures_symbol',
                    'fut_expiration_date', 'futures_close', 'opt_expiration_date', 'strike',
                    'call_put', 'style', 'bid', 'ask', 'settlement', 'volume',
                    'open_interest']
                dfs.append(df_)
                index += 1
            else:
                pass
        df = pd.concat(dfs, axis=0, ignore_index=True)
        df = clean(df)
        _.append(df)
df = pd.concat(_, ignore_index=True)
print("all files are concatenated")
df.to_csv('/Users/dylan/iCollections/桌面文件/Repo/Option_Pricing/Data/Output_data/' + '2020_whole.csv')

file not exists
已合并表0
已合并表1
已合并表2
已合并表3
已合并表4
已合并表5
已合并表6
已合并表7
已合并表8
已合并表9
已合并表10
已合并表11
已合并表12
已合并表13
已合并表14
已合并表15
已合并表16
已合并表17
已合并表18
========== Cleaning Data 1 ==========
2262784 rows are dropped
========== Cleaning Data 2 ==========
3138480 rows are dropped
========== Cleaning Data 3 ==========
525047 rows are dropped
190380 rows remain
file not exists
已合并表0
已合并表1
已合并表2
已合并表3
已合并表4
已合并表5
已合并表6
已合并表7
已合并表8
已合并表9
已合并表10
已合并表11
已合并表12
已合并表13
已合并表14
已合并表15
已合并表16
已合并表17
已合并表18
已合并表19
========== Cleaning Data 1 ==========
2774271 rows are dropped
========== Cleaning Data 2 ==========
3592412 rows are dropped
========== Cleaning Data 3 ==========
686143 rows are dropped
169780 rows remain
file not exists
已合并表0
已合并表1
已合并表2
已合并表3
已合并表4
已合并表5
已合并表6
已合并表7
已合并表8
已合并表9
已合并表10
已合并表11
已合并表12
已合并表13
已合并表14
已合并表15
已合并表16
已合并表17
已合并表18
已合并表19
已合并表20
========== Cleaning Data 1 ==========
2930872 rows are dropped
========== Cleaning Data 2 ==========
3864291 rows are dropped
========== Cleaning Da

In [13]:
df = clean(pd.read_csv('/Users/dylan/iCollections/桌面文件/Repo/Option_Pricing/Data/Output_data/2020_whole.csv'))

========== Cleaning Data 1 ==========
0 rows are dropped
========== Cleaning Data 2 ==========
0 rows are dropped
========== Cleaning Data 3 ==========
0 rows are dropped
2103142 rows remain


## 加入对应 利率 r

In [34]:
r_data = pd.read_excel('/Users/dylan/iCollections/桌面文件/Repo/Option_Pricing/Data/r_data/Treasury-Bill.xlsx')
r_data.columns = ['Date', '1M', '3M', '6M', '12M']

In [31]:
r_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5022 entries, 0 to 5021
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    5022 non-null   datetime64[ns]
 1   1M      5022 non-null   float64       
 2   3M      5022 non-null   float64       
 3   6M      5022 non-null   float64       
 4   12M     5002 non-null   float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 196.3 KB


In [36]:
df

,Unnamed: 0,Date,underlying,exchange,root_symbol,futures_symbol,fut_expiration_date,futures_close,opt_expiration_date,strike,call_put,style,bid,ask,settlement,volume,open_interest
0,0,2020-02-05,NG,NYMEX,NG,NG/20X.NX,10/28/2020,2.237,2020-10-27,2.25,p,A,0.18,0.0,0.2090,5,6
1,1,2020-02-05,NG,NYMEX,LN,NG/20X.NX,10/28/2020,2.237,2020-10-27,2.50,c,E,0.00,0.0,0.1068,575,1275
2,2,2020-02-05,NG,NYMEX,LN,NG/20X.NX,10/28/2020,2.237,2020-10-27,2.55,c,E,0.00,0.0,0.0941,5,24
3,3,2020-02-05,NG,NYMEX,LN,NG/20X.NX,10/28/2020,2.237,2020-10-27,2.75,c,E,0.00,0.0,0.0562,200,1973
4,4,2020-02-05,NG,NYMEX,LN,NG/20X.NX,10/28/2020,2.237,2020-10-27,3.00,c,E,0.00,0.0,0.0302,75,6765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103137,2103137,2020-09-28,MNQ,CME,MQ1,MNQ/20Z.CM,12/18/2020,11394.750,2020-10-02,11520.00,c,E,0.00,0.0,85.5000,16,16
2103138,2103138,2020-09-28,MNQ,CME,MQ1,MNQ/20Z.CM,12/18/2020,11394.750,2020-10-02,11530.00,c,E,0.00,0.0,81.7500,2,2
2103139,2103139,2020-09-28,MNQ,CME,MQ1,MNQ/20Z.CM,12/18/2020,11394.750,2020-10-02,11720.00,c,E,0.00,0.0,28.2500,1,1
2103140,2103140,2020-09-28,MNQ,CME,MQ1,MNQ/20Z.CM,12/18/2020,11394.750,2020-10-02,11800.00,c,E,0.00,0.0,16.7500,1,4


In [38]:
df = pd.merge(df, r_data, on='Date')

In [39]:
df.to_csv('/Users/dylan/iCollections/桌面文件/Repo/Option_Pricing/Data/Output_data/' + '2020_whole.csv')